In [1]:
import numpy as np
import math

In [16]:
class chunk:
  def __init__(self, x_pos, y_pos, z_pos, no_source, size):
    self.x_pos = x_pos #The x-position of the bottom back left corner.
    self.y_pos = y_pos
    self.z_pos = z_pos
    self.no_source = no_source #Have we found that there is no source
    self.size = size #The length, width, and height of the cube

  def source_disproved(self):
    return self.source

In [14]:
x_dimension = 1000 #The number of pieces we are breaking the room into
y_dimension = 1000
z_dimension = 1000
chunk_size = 100 #the number of pieces on each side of each chunk

acceptable_angle = np.pi / 4 #The maximum angle between the line to a source and where the source actually is.

room = np.zeros(shape = (x_dimension, y_dimension, z_dimension))
chunk_space = [[[None]*(int(x_dimension/chunk_size))]*(int(y_dimension/chunk_size))]*(int(y_dimension/chunk_size))

for i in range(0, (int(x_dimension/chunk_size))):
  for j in range(0, (int(y_dimension/chunk_size))):
    for k in range(0, (int(z_dimension/chunk_size))):
      chunk_space[i][j][k] = chunk(i*100, j*100, k*100, False, size = chunk_size)

In [17]:
def get_angle(robot_location, source_location):
  dx = source_location[0] - robot_location[0]
  dy = source_location[1] - robot_location[1]
  dz = source_location[2] - robot_location[2]
  r = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)

  mu = dz / r
  if dx != 0:
    theta = np.arctan(dy/dx)
  else:
    theta = np.pi / 2

  return [mu, theta]

def create_lines(position, angle, room_state):
  """
  This function will return the room matrix with 1s in all of the possible locations for radiation sources
    (i.e. all of the spaces where a radiation source could be to give the angle at the position)

  Inputs:
    position - a vector containing the x, y, and z position of the robot
    angle - a vector containing the mu and gamma for the angle from the robot to the source (with straight ahead being <0,pi/2> and going counter-clockwise)
    room_state - the 3d room matrix, which is updated and returned.
  """
  mu = angle[0]
  gamma = angle[1]

  step_x = np.sqrt(1 - mu ** 2) * np.cos(gamma) #defines the x component of the unit vector
  step_y = np.sqrt(1 - mu ** 2) * np.sin(gamma) #defines the y component of the unit vector
  step_z = mu                                   #defines the z component of the unit vector

  x_coordinate = position[0]
  y_coordinate = position[1]
  z_coordinate = position[2]

  in_bounds = True #
  while in_bounds:
    x_coordinate += step_x
    y_coordinate += step_y
    z_coordinate += step_z

    room_state[math.ceil(x_coordinate)][math.ceil(y_coordinate)][math.ceil(z_coordinate)] += 1

    if abs(x_coordinate - 500) > 498: #if the latest position is too close to the walls, call off the whole thing (this is bad code to avoid index errors)
      in_bounds = False
    if abs(y_coordinate - 500) > 498:
      in_bounds = False
    if abs(z_coordinate - 500) > 498:
      in_bounds = False

  return room_state

def angle_to_chunks(position, relevant_chunk):

  my_x = relevant_chunk.x_pos + (relevant_chunk.size/2)
  my_y = relevant_chunk.y_pos + (relevant_chunk.size/2)
  my_z = relevant_chunk.z_pos + (relevant_chunk.size/2)
  dx = my_x - position[0]
  dy = my_y - position[1]
  dz = my_z - position[2]
  r = np.sqrt(dx ** 2 + dy ** 2 + dz ** 2)
  mu = dz / r
  if dx != 0:
    theta = np.arctan(dy/dx)
  else:
    theta = np.pi / 2
  return (mu, theta)

def remove_chunks(source_angle, my_chunks, position):
  for this_chunk in my_chunks:
    (mu, theta) = angle_to_chunks(position, this_chunk)
    x_chunk_angle = np.sin(mu) * np.cos(theta)
    y_chunk_angle = np.sin(mu) * np.sin(theta)
    z_chunk_angle = np.cos(mu)

    x_source_angle = np.sin(source_angle[0]) * np.cos(source_angle[1])
    y_source_angle = np.sin(source_angle[0]) * np.sin(source_angle[1])
    z_source_angle = np.cos(source_angle[0])

    numerator = (x_chunk_angle*x_source_angle) * (y_chunk_angle*y_source_angle) * (z_chunk_angle*z_source_angle)
    denomenator = ((x_chunk_angle**2)+(y_chunk_angle**2)+(z_chunk_angle**2)) * ((x_source_angle**2)+(y_source_angle**2)+(z_source_angle**2))
    angle = np.arccos(numerator / denomenator)

    if angle > acceptable_angle:
      this_chunk.no_source = True
  pass